In [ ]:
## Some functions from nnUNET
def load_nifty(directory, example_id, suffix):
    """
    Loads a nifti file from the given (directory)
    with the given example_id and suffix
    """
    return nibabel.load(os.path.join(directory, example_id + "_" + suffix + ".nii.gz"))

def load_channels(d, example_id):
    """
    Loads four nifti files from the given directory (d)
    with the given (example_id )
    and the suffixes "flair", "t1", "t1ce", "t2"
    """
    return [load_nifty(d, example_id, suffix) for suffix in ["t1c", "t1n", "t2f", "t2w"]]

In [ ]:
# Some fx based on data exploration
def load_modalities(modalities, data_dir):
    # Create paths for each modality 
    img_modalitypth = [(os.path.join(data_dir + img_folder, img_folder + f"-{m}.nii.gz")) for m in modalities]
    ## double check for files loaded
    for i in range(len(img_modalitypth)) : print(os.path.basename(img_modalitypth[i]))  #double check files loaded
    
    # load modalities into a list, generate headers
    img_modality = []
    hdrs = {}
    for idx, mname in enumerate(modalities):
        globals()[f'{mname}_img'] = nib.load(img_modalitypth[idx])
        img_modality.append(globals()[f'{mname}_img'])
        hdrs[f'{mname}_img'] = img_modality[idx].header
        print(f"Dimensions for modality {mname} is {img_modality[idx].shape}, with isotropic resolution of {hdrs[f'{mname}_img'].get_zooms()} ")
    return img_modality, hdrs


## GANDLF PREPROC
Intensity harmonization: GaNDLF provides multiple normalization and rescaling options to ensure intensity-level harmonization of the entire cohort. Some examples include:
- normalize: simple Z-score normalization
- normalize_positive: this performs z-score normalization only on pixels > 0
- normalize_nonZero: this performs z-score normalization only on pixels != 0
- normalize_nonZero_masked: this performs z-score normalization only on the region defined by the ground truth annotation
- rescale: simple min-max rescaling, sub-parameters include in_min_max, out_min_max, percentiles; this option is useful to discard outliers in the intensity distribution
- Template-based normalization: These options take a target image as input (defined by the target sub-parameter) and perform different matching strategies to match input image(s) to this target.
- histogram_matching: this performs histogram matching as defined by this paper.
    - If the target image is absent, this will perform global histogram equalization.
    - If target is adaptive, this will perform adaptive histogram equalization.
- Resolution harmonization: GaNDLF provides multiple resampling options to ensure resolution-level harmonization of the entire cohort. Some examples include:
- resample: resamples the image to the specified by the resolution sub-parameter
- resample_min: resamples the image to the maximum spacing defined by the resolution sub-parameter; this is useful in cohorts that have varying resolutions, but the user wants to resample to the minimum resolution for consistency
- resize_image: NOT RECOMMENDED; resizes the image to the specified size
- resize_patch: NOT RECOMMENDED; resizes the extracted patch to the specified size